In [1]:
import pandas as pd
import numpy as np
from dateutil import parser
from datetime import timedelta
from datetime import date
from datetime import datetime
from datetime import time
from IPython.display import clear_output
from tqdm import tnrange, tqdm_notebook
from time import sleep

In [ ]:
print(datetime.fromtimestamp(1530749280))

In [ ]:
d = date(2018, 7, 1)
print(d)
t = parser.parse("16:30").time()
print(t)
dt =  datetime.combine(d, t)
print(dt)
dt = dt + timedelta(minutes=2400)
print(dt)
print(dt.timestamp() + 10*60)

In [2]:
df = pd.read_csv('stops_all.csv')
train_id_list = df.train_id.unique()

In [3]:
df_stations = pd.read_csv('stations_all.csv')
station_id_list = df_stations.id.unique()
df_stations[':LABEL'] = "Station"
df_stations.columns = ['station_id:ID(Station-ID)', 'name', ':LABEL']
df_stations.to_csv('graph_stations.csv', index=False, encoding='utf-8')

In [4]:
df_trains = pd.read_csv('trains_all.csv')
df_trains = df_trains.drop(['description','travel_time'], 1)
df_trains[':LABEL'] = "Train"
df_trains.columns = ['train_id:ID(Train-ID)', 'number', 'name', ':LABEL']
df_trains.to_csv('graph_trains.csv', index=False, encoding='utf-8')

In [5]:
day_list = []
start_day = date(2018, 7, 1)
for x in range(0, 7):
    day = start_day+timedelta(days=x)
    day_list.append(day)
print(day_list)

[datetime.date(2018, 7, 1), datetime.date(2018, 7, 2), datetime.date(2018, 7, 3), datetime.date(2018, 7, 4), datetime.date(2018, 7, 5), datetime.date(2018, 7, 6), datetime.date(2018, 7, 7)]


In [6]:
def getTimestamp(date_from, time_from, travel_minutes):
    t = parser.parse(time_from).time()
    dt =  datetime.combine(date_from, t)
    dt = dt + timedelta(minutes=travel_minutes)
    return dt.timestamp()

In [ ]:
getTimestamp(date(2018,7,1), "16:30", 2400)

In [7]:
def timeDiff(time_from, time_to, prev_minutes):
    prev_hours = prev_minutes // 60
    td = parser.parse(time_to) - parser.parse(time_from)
    hours = td.total_seconds() // 3600
    #print(hours)
    #print(hours - prev_hours)
    if hours > 12 :
        #print("-12h")
        td = td - timedelta(hours=12)
    if hours < prev_hours:
        #print("+24h")
        td = td + timedelta(hours=24)
        #print(td)
        #print(td.total_seconds())
    minutes = td.total_seconds() // 60
    return minutes

In [8]:
def updateGraphForTrainId(train_id):
    df_train = df.loc[df.train_id == train_id].copy()
    for index_from, row_from in df_train.iterrows():
        prev_minutes = 0
        for index_to, row_to in df_train.loc[index_from+1:,].iterrows():
            travel_time = timeDiff(row_from.departure_time, row_to.arrival_time, prev_minutes)            
            for day in day_list:  
                ts_dep = getTimestamp(day, row_from.departure_time, prev_minutes)
                event_dep_id = str(train_id)+":"+str(row_from.station_id)+":"+str(int(ts_dep))+":dep"
                event_dep = (event_dep_id, train_id, row_from.station_id, int(ts_dep), "Event;Departure")
                event_list.append(event_dep)
                
                train_event_dep = (train_id, event_dep_id,"DEPARTED", 0)
                train_event_dep_list.append(train_event_dep)
                
                station_event_dep = (row_from.station_id, event_dep_id,"AT_STATION", 0)
                station_event_dep_list.append(station_event_dep)
                
                #print(str(train_id)+":"+str(row_to.station_id))
                ts_arr = getTimestamp(day, row_to.arrival_time, travel_time)
                # custom feature
                ts_arr = ts_arr + 10*60
                event_arr_id = str(train_id)+":"+str(row_to.station_id)+":"+str(int(ts_arr))+":arr"
                event_arr = (event_arr_id, train_id, row_to.station_id, int(ts_arr), "Event;Arrival")
                event_list.append(event_arr)
                
                train_event_arr = (event_arr_id, train_id,"ARRIVED", 0)
                train_event_arr_list.append(train_event_arr)
                
                station_event_arr = (event_arr_id, row_to.station_id,"AT_STATION", 0)
                station_event_arr_list.append(station_event_arr)
                
                trip = (event_dep_id, train_id, int(travel_time + 10), event_arr_id, "TRAVEL_TO")
                trip_list.append(trip)
            prev_minutes = travel_time
    return

In [9]:
trip_list = []
event_list = []
station_event_dep_list = []
station_event_arr_list = []
train_event_dep_list = []
train_event_arr_list = []
for train_id in train_id_list: 
    updateGraphForTrainId(train_id)
event_list = list(set(event_list))

In [11]:
columns = ['event_id:ID(Event-ID)', 'train_id(Train-ID)', 'station_id(Station-ID)', 'timestamp:double', ':LABEL']
df_events = pd.DataFrame(event_list, columns=columns)
df_events.to_csv('graph_events.csv', index=False)

In [10]:
columns = [':START_ID(Event-ID)', 'train_id(Train-ID)', 'travel_time:int', ':END_ID(Event-ID)', ':TYPE']
df_trips = pd.DataFrame(trip_list, columns=columns)
df_trips.to_csv('graph_trips.csv', index=False)

In [19]:
columns = [':START_ID(Event-ID)', ':END_ID(Station-ID)', ':TYPE', 'travel_time:int']
df_station_events = pd.DataFrame(station_event_arr_list, columns=columns)
df_station_events.to_csv('graph_station_events_arr.csv', index=False)

columns = [':START_ID(Station-ID)', ':END_ID(Event-ID)', ':TYPE', 'travel_time:int']
df_station_events = pd.DataFrame(station_event_dep_list, columns=columns)
df_station_events.to_csv('graph_station_events_dep.csv', index=False)

In [20]:
columns = [':START_ID(Event-ID)', ':END_ID(Train-ID)', ':TYPE', 'travel_time:int']
df_train_events = pd.DataFrame(train_event_arr_list, columns=columns)
df_train_events.to_csv('graph_train_events_arr.csv', index=False)

columns = [':START_ID(Train-ID)', ':END_ID(Event-ID)', ':TYPE', 'travel_time:int']
df_train_events = pd.DataFrame(train_event_dep_list, columns=columns)
df_train_events.to_csv('graph_train_events_dep.csv', index=False)

In [ ]:
'true' != 'yes'

In [ ]:
wait_list = []
stat_left = len(station_id_list)
from_left = 0
from_iter = 0
to_left = 0
wait_cnt = 0

#pbar1 = tqdm_notebook(total=100, desc='1st loop')
#for i in range(10):
#    pbar1.update(10)
#pbar1.close()

for station_id in tqdm_notebook(station_id_list, desc='Stations'):
    #print(station_id)
    df_station = df_events.loc[df_events['station_id(Station-ID)'] == station_id].copy()
    #print(len(df_station))
    df_station = df_station.sort_values(by=['timestamp:double']).reset_index(drop=True)
    from_left = len(df_station)
    from_iter = 1
    #clear_output()
    #print("stat_left: "+str(stat_left)+" "+"from_left: "+str(from_left)+" "+"to_left: "+str(to_left)+" "+"wait_cnt: "+str(wait_cnt)+" ")    
    pBarFrom = tqdm_notebook(total=len(df_station), desc='From', leave=False)
    for index_from, row_from in df_station.iterrows():
        pBarFrom.update(1)        
        if('Event;Arrival' == row_from[':LABEL']):
            to_left = len(df_station) - from_iter
            #clear_output()
            #print("stat_left: "+str(stat_left)+" "+"from_left: "+str(from_left)+" "+"to_left: "+str(to_left)+" "+"wait_cnt: "+str(wait_cnt)+" ")            
            pBarTo = tqdm_notebook(total=to_left, desc='To', leave=False)
            for index_to, row_to in df_station.loc[index_from+1:,].iterrows():                
                pBarTo.update(1)
                if('Event;Departure' == row_to[':LABEL'] and (row_to['train_id(Train-ID)'] != row_from['train_id(Train-ID)'])):
                    wait_time = row_to['timestamp:double'] - row_from['timestamp:double']
                    if(wait_time > 0):
                        if(wait_time < 60*60*3):
                            event_from = row_from['event_id:ID(Event-ID)']
                            event_to = row_to['event_id:ID(Event-ID)']
                            wait_cnt += 1
                            wait = (event_from, event_to, int(wait_time), "WAITING")
                            wait_list.append(wait)
                            #clear_output()
                            #print("stat_left: "+str(stat_left)+" "+"from_left: "+str(from_left)+" "+"to_left: "+str(to_left)+" "+"wait_cnt: "+str(wait_cnt)+" ")
                        else:
                            #print("more then 3 hours, from: "+str(row_from['timestamp:double'])+" "+"to: "+str(row_to['timestamp:double']))
                            pBarTo.update(to_left)
                            break
                    #else:
                        #print("less then 0, from: "+str(row_from['timestamp:double'])+" "+"to: "+str(row_to['timestamp:double']))
                #else:
                    #print("type_to: "+str(row_to[':LABEL']) + " trains equal: "+str(row_to['train_id(Train-ID)'] == row_from['train_id(Train-ID)']))
                to_left -= 1
                #clear_output()
                #print("stat_left: "+str(stat_left)+" "+"from_left: "+str(from_left)+" "+"to_left: "+str(to_left)+" "+"wait_cnt: "+str(wait_cnt)+" ")
            pBarTo.close()
        #else:
        #    print("type_from: "+str(row_from[':LABEL']))            
        from_left -= 1
        from_iter += 1
        #clear_output()
        #print("stat_left: "+str(stat_left)+" "+"from_left: "+str(from_left)+" "+"to_left: "+str(to_left)+" "+"wait_cnt: "+str(wait_cnt)+" ")        
    pBarFrom.close()
    stat_left -= 1
    #clear_output()
    #print("stat_left: "+str(stat_left)+" "+"from_left: "+str(from_left)+" "+"to_left: "+str(to_left)+" "+"wait_cnt: "+str(wait_cnt)+" ")

In [ ]:
columns = [':START_ID(Event-ID)', ':END_ID(Event-ID)', 'wait_time:int', ':TYPE']
df_waits = pd.DataFrame(wait_list, columns=columns)
df_waits.to_csv('graph_waits.csv', index=False)